# Improving Your Prompt By Thinking

In this notebook, we focus exclusively on refining and optimizing a single prompt to achieve the desired output from language models. Rather than creating complex systems or chains of prompts, we'll apply an iterative improvement approach to a single prompt.

## Set up the enviornment

In [1]:
!pip install -U sagemaker boto3 botocore json_repair

Please ensure you're using boto3 version 1.37.1 or higher for this code to work correctly.

In [2]:
import boto3
print(f"boto3 version: {boto3.__version__}")

boto3 version: 1.37.11


In [3]:
import json
import boto3
import time
import json
import traceback
import os
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from collections import defaultdict
import re
from IPython.display import display, Code
from json_repair import repair_json
import copy
import pandas as pd

## Write helper fuctions
Our code utilizes three key helper functions:

- **call_bedrock_converse**: Handles communication with AWS Bedrock models, sending prompts and receiving AI responses while managing authentication and API interactions.
- **load_json_from_llm_result**: Extracts and parses JSON data from the LLM's response text, cleaning any formatting issues to ensure valid JSON structure for further processing.

In [4]:
def call_bedrock_converse(prompt, model_id, temperature=0.7, top_p=250, max_tokens=4096):
    """
    Call Amazon Bedrock using the Converse API to generate a response.
    
    Args:
        prompt (str): The prompt to send to the model
        model_id (str): The model ID (e.g., "anthropic.claude-3-sonnet-20240229-v1:0")
        temperature (float): Controls randomness (0-1)
        top_k (int): Limits token selection to top K options
        max_tokens (int): Maximum tokens to generate
        
    Returns:
        dict: The model's response
    """
    # Initialize Bedrock client
    bedrock_runtime = boto3.client(
        service_name="bedrock-runtime",
    )
    
    # Make the API call
    response = bedrock_runtime.converse(
        modelId=model_id,
        messages= [
            {
                "role": "user",
                "content": [
                    {"text": prompt}
                ]
            }
        ],
        inferenceConfig= {
            "temperature": temperature,
            "maxTokens": max_tokens
        }
    )
    
    output_message = response['output']['message']

    return "\n".join(x["text"] for x in output_message["content"])

In [5]:
def load_json_from_llm_result(text):
    """
    Extract and clean JSON from markdown code blocks.
    Returns the first valid JSON found or None if no valid JSON is found.
    """
    # First, try to find JSON blocks
    pattern = r"```(?:json)?\s*([\s\S]*)```"
    matches = re.findall(pattern, text, re.DOTALL)
    if not matches:
        return None
    # Process each potential JSON block
    for json_text in matches:
        print(json_text)
        good_json_string = repair_json(json_text)
        # Try to parse the JSON to verify it's valid
        try:
            return json.loads(good_json_string)
        except json.JSONDecodeError:
            continue
    # If we've tried all matches and none are valid JSON, return None
    return None

## Build a baseline 

Set Baseline Prompt and Target Model



You can have a pompt like
```
baseline_prompt = """
You are a helpful AI assistant. Answer the user's questions accurately, 
truthfully, and to the best of your ability based on the information 
available to you.
"""
```

When testing prompt variations, having this baseline allows you to quantify exactly how changes affect model performance, response quality, and adherence to specific requirements.

If you already have one, 

### Load out sample dataset

In this notebook, we provide a sample test cases file named `test_cases.json`. This file includes:

- The primary prompt to be used with the model `prompt_template`
- A collection of test cases in the test_cases field, where each test case contains:
    - A user question
    - The ground truth response 


```json
{
  "prompt": "You are a helpful assistant. Answer questions accurately and concisely.",
  "test_cases": [
    {
      "user_question": "I need my secret code changed for the plastic rectangle I use at the money machine, and while you're at it, I want to make sure my mobile number is up to date so I get those little messages when I use it.",
      "ground_truth": "PIN_RESET"
    },
    {
      "user_question": "I noticed my digital banking access isn't working and I keep getting a message about verification failing. I already tried the reset link, but it didn't come to my email. By the way, I also got this strange text claiming to be from your bank asking for my details.",
      "ground_truth": "ESCALATION"
    },
  ]
}
```

In [6]:
# Read data from a JSON file
with open('./src/data/test_cases.json', 'r') as file:
    test_cases = json.load(file)

In [7]:
# AWS target model ID
target_model_id = "us.amazon.nova-pro-v1:0"

### First batch of test
For each test case in our test_cases.json file:

- We extract the user question from the test case
- We call the Bedrock API with this question using the call_bedrock_converse function
- The model generates a response based on the provided prompt and question

We receive and store the model's answer for comparison with the expected response

In [8]:
from string import Template

def process_single_test_case(test_case, prompt_template, target_model_id, case_idx, temperature=0.1, top_p=0.9, max_tokens=2000):
    """
    Process a single test case and return the result
    
    Args:
        test_case (dict): The test case to process
        prompt_template (str): Template string with {user_question} placeholder
        target_model_id (str): Model ID to use for inference
        case_idx (int): Case index for tracking
        temperature (float): Temperature setting for inference
        top_p (float): Top-p setting for inference
        max_tokens (int): Maximum tokens to generate
        
    Returns:
        dict: The processed test case result
    """
    user_question = test_case.get("user_question", "")
    groundtruth_result = test_case.get("ground_truth", "")
    generated_text = ""

    try:
        # Format the prompt template with the user question        
        template = Template(prompt_template)
        formatted_prompt = template.safe_substitute(user_question=user_question)

        # Call the Bedrock Converse API
        generated_text = call_bedrock_converse(
            prompt=formatted_prompt,
            model_id=target_model_id,
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens
        )
        #print("############################")
        #print("Prompt: ")
        #print(formatted_prompt)
        #print("Prediction: ")
        #print(generated_text)
        #print("############################")

        results_llm = load_json_from_llm_result(generated_text)
        
        # Create result entry
        case_result = {
            "user_question": user_question,
            "ground_truth": groundtruth_result,
            "prediction": results_llm["prediction"],
            "explanation": results_llm["explanation"],
            "case_type": "llm_success" 
        }
        
    except Exception as e:
        # Handle errors
        error_trace = traceback.format_exc()
        case_result = {
            "user_question": user_question,
            "ground_truth": groundtruth_result,
            "prediction": "Error",
            "explanation": "Original generated text: " +  generated_text,
            "case_type": "llm_error"
        }
        
        print(f"\nError in test case {case_idx+1}:")
        print(error_trace)
    
    # Add metadata for visualization
    case_result.update({
        "case_idx": case_idx + 1
    })
    
    return case_result

In [9]:
def evaluate_test_results(result_data):
    """
    Evaluate test results by comparing predictions with ground truth.
    
    Args:
        result_data (dict): Dictionary containing test cases and their results
        
    Returns:
        dict: Updated result_data with evaluation metrics
    """
    task_success = 0
    for index, test_case in enumerate(result_data['test_cases']):
        expected_output = test_case.get("ground_truth", "")
        llm_output = test_case.get("prediction", "")
        
        if expected_output == llm_output:
            task_success += 1
            result_data['test_cases'][index]['task_succeed'] = True
        else:
            result_data['test_cases'][index]['task_succeed'] = False
            
    result_data['stats']['task_succeed'] = task_success
    return result_data

In [10]:
def execute_test_cases(data, target_model_id, output_file=None):
    """
    Execute all test cases and track results
    
    Args:
        data (dict): Data containing prompt template and test cases
        target_model_id (str): Model ID to use for inference
        output_file (str, optional): Path to save results. If None, results aren't saved.
        
    Returns:
        dict: Results of all test cases with statistics
    """
    # Initialize counters and data structures
    prompt_template = data.get("prompt_template", "")
    test_cases = data.get("test_cases", [])
    total_cases = len(test_cases)
    
    suite_results = {
        "prompt_template": prompt_template,
        "test_cases": [],
        "stats": {"total": total_cases, "llm_successful": 0, "llm_fail": 0,"task_succeed": 0},
    }

    # Set up progress bar
    with tqdm(total=total_cases, desc="Processing Test Cases") as pbar:
        # Process each test case
        for case_idx, test_case in enumerate(test_cases):
            # Process the test case
            case_result = process_single_test_case(
                test_case,
                prompt_template,
                target_model_id,
                case_idx,
            )

            # Add case result to results
            suite_results["test_cases"].append(case_result)
            
            # Update statistics
            if case_result["case_type"] == "llm_success":
                suite_results["stats"]["llm_successful"] += 1
            else:
                suite_results["stats"]["llm_fail"] += 1

            # Update progress bar
            pbar.update(1)
            pbar.set_postfix({
                "Success": f"{suite_results['stats']['llm_successful']}/{total_cases}",
            })
    # Evaluate task success (comparing predictions with ground truth)
    suite_results = evaluate_test_results(suite_results)
    
    # Save results if output file is specified
    if output_file:
        # Ensure directory exists
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        
        with open(output_file, "w") as f:
            json.dump(suite_results, f, indent=2)
        print(f"Results saved to {output_file}")

    return suite_results


In [11]:
# Create results directory if it doesn't exist
results_dir = "results"
os.makedirs(results_dir, exist_ok=True)
    
# Create output filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = os.path.join(results_dir, f"test_results_{timestamp}.json")
    
# Initialize counters and results structures
baseline_result = execute_test_cases(test_cases, target_model_id, output_file)

Processing Test Cases:   4%|▍         | 1/25 [00:01<00:30,  1.26s/it, Success=1/25]

{
  "prediction": "MULTI_ISSUE",
  "explanation": "The inquiry contains two separate requests: one for changing the PIN (classified as PIN_RESET) and another for updating the contact information (classified as CONTACT_INFO_UPDATE). Since it involves multiple issues, it doesn't fit neatly into a single category."
}



Processing Test Cases:   8%|▊         | 2/25 [00:02<00:25,  1.11s/it, Success=2/25]

{
  "prediction": "ESCALATION",
  "explanation": "The inquiry involves multiple issues: failed verification, a missing reset link, and a suspicious text message. These indicate potential security concerns that need to be addressed by higher-level support."
}



Processing Test Cases:  12%|█▏        | 3/25 [00:02<00:20,  1.10it/s, Success=3/25]

...

{
  "prediction": "IN_SCOPE",
  "explanation": "The inquiry is related to providing documentation for financial transactions, which falls under the scope of financial services."
}



Processing Test Cases:  96%|█████████▌| 24/25 [00:23<00:00,  1.04it/s, Success=24/25]

{
  "prediction": "IN_SCOPE",
  "explanation": "The inquiry is about a sudden drop in the customer's credit score, which is directly related to the financial services provided by the company. It falls under questions about our financial services."
}



Processing Test Cases: 100%|██████████| 25/25 [00:24<00:00,  1.04it/s, Success=25/25]

{
  "prediction": "OUT_OF_SCOPE",
  "explanation": "The inquiry appears to be nonsensical and does not relate to any specific category of customer service issues or financial services topics."
}

Results saved to results/test_results_20250312_093709.json


### Verify the result with the expected result


Define a comparison function that evaluates how closely your output matches the expected result (ground truth). This function should:

- Accept both your calculated result and the ground truth as parameters
- Return a clear indication of how well the results match

In [12]:
# Process each test case within the suite
task_success = 0
for index, test_case in enumerate(baseline_result['test_cases']):
    # Evaluate with compare_result    
    if baseline_result['test_cases'][index]['task_succeed'] == True:
        task_success +=1

print("Successful: ", task_success, "of total number of", len(baseline_result['test_cases']))

Successful:  13 of total number of 25


## Prompt improvement with the thinking process


### What is reasoning model

Anthropic's Claude 3.7 Sonnet, now available on Amazon Bedrock, represents a groundbreaking advancement in generative AI as the first hybrid reasoning model. This state-of-the-art model combines rapid response capabilities with extended, step-by-step reasoning, allowing users to toggle between modes based on task complexity. In standard mode, it delivers quick and efficient answers, while extended thinking mode enables detailed problem-solving through logical analysis and self-reflection. 



In [13]:
import boto3
import json
from botocore.config import Config
from IPython.display import display, HTML

def error_analysis_with_reasoning(prompt, temperature=1, max_tokens=8192, 
                                 thinking_budget=4096, system_prompt="", 
                                 model_id= " "):
    """
    Call Amazon Bedrock using the Converse API with thinking capability
    
    Args:
        prompt (str): The prompt to send to the model
        temperature (float): Controls randomness (0-1)
        top_p (int): Limits token selection to top P options
        max_tokens (int): Maximum tokens to generate
        thinking_budget (int): Maximum tokens to think
        system_prompt (str): System prompt to guide the model
        model_id (str): The model ID
        
    Returns:
        dict: The model's response with thinking and other content
    """



    # Initialize Bedrock client
    config = Config(
         connect_timeout=300,
         read_timeout=300
    )
    bedrock_runtime = boto3.client(service_name='bedrock-runtime', config=config)

    # Format system prompt as required by Converse API
    formatted_system_prompt = [{"text": system_prompt}] if system_prompt else []
    
    # Format the message for Converse API
    messages = [
        {
            "role": "user",
            "content": [{"text": prompt}]
        }
    ]
    
    # Configure inference parameters
    inference_config = {
        "temperature": temperature,
        "maxTokens": max_tokens
    }


    if "sonnet" in model_id:
        # Configure reasoning parameters
        reasoning_config = {
            "thinking": {
                "type": "enabled",
                "budget_tokens": thinking_budget
            },
        }

        # Make the API call
        response = bedrock_runtime.converse(
            modelId=model_id,
            messages=messages,
            system=formatted_system_prompt,
            inferenceConfig=inference_config,
            additionalModelRequestFields=reasoning_config
        )
    elif "deepseek" in model_id: 
        # Make the API call
        response = bedrock_runtime.converse(
            modelId=model_id,
            messages=messages,
            inferenceConfig=inference_config,
            system=formatted_system_prompt,
        )

    # Initialize result dictionary
    result = {}
    
    # Extract content blocks using the exact pattern provided
    content_blocks = response["output"]["message"]["content"]
    
    reasoning = None
    text = None
    
    # Process each content block to find reasoning and response text
    for block in content_blocks:
        if "reasoningContent" in block:
            reasoning = block["reasoningContent"]["reasoningText"]["text"]
        if "text" in block:
            text = block["text"]
    
    # Add the extracted contents to the result dictionary
    if reasoning:
        result['reasoning'] = reasoning
        display(HTML("<h5>REASONING</h5>"))
        display(HTML(f"<pre style='white-space:pre-wrap;'>{reasoning}</pre>"))
    
    if text:
        result['text'] = text
        display(HTML("<h5>RESPONSE</h5>"))
        display(HTML(f"<pre style='white-space:pre-wrap;'>{text}</pre>"))
        
    # Add token usage information to result
    if 'usage' in response:
        result['token_usage'] = response['usage']
    
    return result

In [14]:
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
#model_id = "us.deepseek.r1-v1:0"

thinking = error_analysis_with_reasoning("what is the solution for x+1 =10",max_tokens=2048, thinking_budget=1024,model_id=model_id)

### Revise template with reasoning model

Let's try to use Claude 3.7 Sonnet to revise our prompt with a "thinking" approach.  The revised prompt will encourage Claude to pause, reflect, and methodically address complex questions before formulating its final response.

In [15]:
critique_prompt_template = """
Analyze the classification performance and provide detailed reasoning for prompt improvements:

Current Template:
<current_template>
${input_current_template}
</current_template>

Evaluation Results:
<evaluation_results>
${evaluation_results}
</evaluation_results>


Follow these thinking steps in order:

1. STEP 1 - Error Pattern Analysis:
   - List each misclassified case
   - Group similar errors
   - Focus on how the prompt's instructions led to these errors

2. STEP 2 - Prompt-Specific Root Cause Investigation:
   For each error pattern identified above, analyze:
   - Which parts of the current prompt led to misinterpretation?
   - Are there ambiguous or missing instructions?
   - Are the classification criteria clearly defined?
   - Is the format/structure of the prompt causing confusion?

3. STEP 3 - Historical Context:
   Previous Iterative Suggestions: 
   <suggestion_history>
   ${suggestion_history}        
   </suggestion_history>

   Analyze only prompt-related changes:
   - Which prompt modifications were effective/ineffective?
   - Which instruction clarity issues persist?
   - What prompt elements still need refinement?
   - Focus more on recent iterations 

4. STEP 4 - Prompt Improvement Ideas:
   Suggest only changes to prompt instructions and structure:
   - Clearer classification criteria
   - Better examples or explanations
   - More precise instructions
   - Better prompt structure or organization
   - Specific wording improvements
  
   AVOID suggesting:
   Adding more training data
   Modifying the model
   Changes to the underlying AI system
   Adding new model capabilities
   Adding directly the evalution samples into the suggesting

   
   Base on the Current Template between <current_template> </current_template>
   
   Output your final improvement suggestions between <suggestion> </suggestion> 

"""

In [16]:
from string import Template
template = Template(critique_prompt_template)
current_critique_prompt = template.substitute(
    input_current_template = baseline_result['prompt_template'], 
    evaluation_results= json.dumps(baseline_result['test_cases']), 
    suggestion_history = " "
)
display(Code(current_critique_prompt, language=None))

Analyze the classification performance and provide detailed reasoning for prompt improvements:

Current Template:
<current_template>
You are a Financial Services Assistant. Classify each customer inquiry into one of these categories:
ACKNOWLEDGMENT - For greetings and thanks
PASSWORD_RESET - For password issues
CONTACT_INFO_UPDATE - For updating contact information
PIN_RESET - For PIN number issues
TRANSACTION_STATUS - For questions about pending transactions
AUTHENTICATION_SETUP - For setting up security features
CARD_DISPUTE - For unauthorized charges or transaction problems
ESCALATION - For security issues and complex problems
IN_SCOPE - For questions about our financial services
OUT_OF_SCOPE - For unrelated topics

### User Inquiry:
${user_question}. Output format: Please output directly in json Output format: Respond with a JSON object containing two fields - "prediction" with the category name, and "explanation" with your reasoning for the classification. When you output JSON, ALWAYS start with ```json
</current_template>

Evaluation Results:
<evaluation_results>
[{"user_question": "I need my secret code changed for the plastic rectangle I use at the money machine, and while you're at it, I want to make sure my mobile number is up to date so I get those little messages when I use it.", "ground_truth": "PIN_RESET", "prediction": "MULTI_ISSUE", "explanation": "The inquiry contains two separate requests: one for changing the PIN (classified as PIN_RESET) and another for updating the contact information (classified as CONTACT_INFO_UPDATE). Since it involves multiple issues, it doesn't fit neatly into a single category.", "case_type": "llm_success", "case_idx": 1, "task_succeed": false}, {"user_question": "I noticed my digital banking access isn't working and I keep getting a message about verification failing. I already tried the reset link, but it didn't come to my email. By the way, I also got this strange text claiming to be from your bank asking for my details.", "ground_truth": "ESCALATION", "prediction": "ESCALATION", "explanation": "The inquiry involves multiple issues: failed verification, a missing reset link, and a suspicious text message. These indicate potential security concerns that need to be addressed by higher-level support.", "case_type": "llm_success", "case_idx": 2, "task_succeed": true}, {"user_question": "Hey! Just wondering what the status is on that application I submitted last week? The website still says 'processing' and I need to know if I should look elsewhere.", "ground_truth": "IN_SCOPE", "prediction": "TRANSACTION_STATUS", "explanation": "The customer is inquiring about the status of an application they submitted, which falls under questions about pending transactions.", "case_type": "llm_success", "case_idx": 3, "task_succeed": false}, {"user_question": "Someone keeps trying to withdraw cash using my account. There are three failed attempts at different ATMs across town yesterday. I still have my card but I'm traveling in Europe right now.", "ground_truth": "ESCALATION", "prediction": "CARD_DISPUTE", "explanation": "The customer is reporting unauthorized attempts to withdraw cash from their account at different ATMs, which falls under the category of a card dispute due to potential unauthorized use.", "case_type": "llm_success", "case_idx": 4, "task_succeed": false}, {"user_question": "My wife and I just got our electronic statements, and we noticed several small payments to digital services we never signed up for. They're all under 5 dollars so we didn't notice them before. Are these legitimate fees or something else?", "ground_truth": "CARD_DISPUTE", "prediction": "CARD_DISPUTE", "explanation": "The inquiry describes unauthorized small payments on the account, which indicates a potential issue with unauthorized charges or transaction problems.", "case_type": "llm_success", "case_idx": 5, "task_succeed": true}, {"user_question": "The cash machine ate my card yesterday evening after I entered 

In [17]:
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
#model_id = "us.deepseek.r1-v1:0"

feedbacks = error_analysis_with_reasoning(current_critique_prompt,max_tokens=4096, thinking_budget=2048, model_id=model_id)

Now we can create a more structured and comprehensive prompt template that includes system context, model information, guidelines, and example sections.

In [18]:
guidance_prompt_improvement_template = """
You need to improve the Current Template following the Critique Analysis.  

Current Template:
<current_template>
${input_current_template}
</current_template>

Instructions for improved template:
1. Take the Current Template as a base. 
2. Incorporate specific improvements identified in the analysis
3. Ensure the new template maintains the basic structure but addresses the identified issues. 
4. The improved template should be a complete, ready-to-use prompt

Critique Analysis: 
<critique_feedbacks>
${critique_feedbacks}
</critique_feedbacks>

When you output JSON, ALWAYS 
Return your response in this exact JSON format, start with 
```json
{
    "root_cause": "Provide the root cause analysis from the feedbacks, please details Error Pattern Analysis and Root Cause Investigation, String FORMAT",
    "improved_template": "Provide the complete new template here with all recommended changes incorporated. This should be a fully functional template ready for the next iteration. String FORMAT"
}

IMPORTANT: The improved_template must be improved veresion o fCurrent Template by incorperating the recommended changes. PLEASE KEEP THE improved_template CONCISE AND EFFECTIVE.
        """

In [19]:
from string import Template
template = Template(guidance_prompt_improvement_template)
current_critique_prompt = template.substitute(
    input_current_template=baseline_result['prompt_template'],
    critique_feedbacks=feedbacks['text']
)
display(Code(current_critique_prompt, language=None))

You need to improve the Current Template following the Critique Analysis.  

Current Template:
<current_template>
You are a Financial Services Assistant. Classify each customer inquiry into one of these categories:
ACKNOWLEDGMENT - For greetings and thanks
PASSWORD_RESET - For password issues
CONTACT_INFO_UPDATE - For updating contact information
PIN_RESET - For PIN number issues
TRANSACTION_STATUS - For questions about pending transactions
AUTHENTICATION_SETUP - For setting up security features
CARD_DISPUTE - For unauthorized charges or transaction problems
ESCALATION - For security issues and complex problems
IN_SCOPE - For questions about our financial services
OUT_OF_SCOPE - For unrelated topics

### User Inquiry:
${user_question}. Output format: Please output directly in json Output format: Respond with a JSON object containing two fields - "prediction" with the category name, and "explanation" with your reasoning for the classification. When you output JSON, ALWAYS start with ```json
</current_template>

Instructions for improved template:
1. Take the Current Template as a base. 
2. Incorporate specific improvements identified in the analysis
3. Ensure the new template maintains the basic structure but addresses the identified issues. 
4. The improved template should be a complete, ready-to-use prompt

Critique Analysis: 
<critique_feedbacks>
# Classification Performance Analysis and Prompt Improvement Recommendations

## Error Pattern Analysis

After reviewing the evaluation results, I identified several recurring misclassification patterns:

1. **Security Issues Misclassification**: Multiple cases where ESCALATION issues were labeled as CARD_DISPUTE (cases 4, 6, 8, 13)
   - Example: "Someone keeps trying to withdraw cash using my account" was labeled CARD_DISPUTE instead of ESCALATION

2. **IN_SCOPE vs. Specific Categories Confusion**: General financial services inquiries being narrowly categorized (cases 3, 7, 16, 18)
   - Example: "Planning to add daughter as authorized user" classified as AUTHENTICATION_SETUP instead of IN_SCOPE

3. **Multiple Issues Handling**: No clear guidance for handling inquiries containing multiple issues (case 1)
   - Example: PIN reset + contact info update request was incorrectly classified as "MULTI_ISSUE" (which isn't in the defined categories)

4. **Urgency/Severity Recognition Failures**: Missing important contextual clues that would elevate an issue (cases 10, 14)
   - Example: Inquiries about deceased person's account not recognized as requiring escalation

## Root Cause Analysis

1. **Ambiguous Category Definitions**:
   - The boundaries between CARD_DISPUTE and ESCALATION aren't clearly defined
   - IN_SCOPE description is too vague ("questions about our financial services")

2. **Missing Priority Framework**:
   - No instructions on which criteria should take precedence when multiple categories apply
   - No guidance on handling multi-issue inquiries

3. **Incomplete Classification Criteria**:
   - No specific indicators for when security issues require escalation
   - No mention of urgency factors that should influence classification

4. **Classification Logic Gaps**:
   - No decision tree or systematic approach to classification
   - No explicit criteria for distinguishing between simple and complex issues

## Prompt Improvement Recommendations

<suggestion>
You are a Financial Services Assistant. Classify each customer inquiry into one of these categories:

ESCALATION - For urgent issues requiring immediate attention, including:
- Suspected fraud or unauthorized account access
- Security breaches or identity theft concerns
- Large unexplained account balance changes
- Inquiries about deceased individuals' accounts
- Multiple failed authentication attempts
- Urgent issues affecting immediate financial access

CARD_DISPUTE - For transaction-specific issues, including:
- Unauthorized or unrecognized charges
- Duplicate transactions
- Incorrect transaction amounts
- Merchandise or 

In [20]:
def improving_prompt_with_feedback(current_critique_prompt):
    target_model_id = "us.amazon.nova-pro-v1:0"
    improvment_results = " "
    try:
        # Call the Bedrock Converse API
        generated_text = call_bedrock_converse(
            prompt=current_critique_prompt,
            model_id=target_model_id,
            temperature=0.1,
            top_p=0.9,
            max_tokens=2048
        )
        print(generated_text)
        results_llm = load_json_from_llm_result(generated_text)
        # Create result entry
        improvment_results = {
            "root_cause": results_llm["root_cause"],
            "improved_template": results_llm["improved_template"],
        }
        
    except Exception as e:
        print(e)
    

    return improvment_results

In [21]:
improved_result = improving_prompt_with_feedback(current_critique_prompt)

```json
{
    "root_cause": "The root cause analysis identified several issues: 1) Security issues were often misclassified as CARD_DISPUTE instead of ESCALATION. 2) General financial inquiries were incorrectly categorized into specific categories instead of IN_SCOPE. 3) There was no guidance for handling inquiries with multiple issues. 4) The template failed to recognize urgency or severity in certain contexts. These issues stemmed from ambiguous category definitions, a missing priority framework, incomplete classification criteria, and gaps in classification logic.",
    "improved_template": "You are a Financial Services Assistant. Classify each customer inquiry into one of these categories:\n\nESCALATION - For urgent issues requiring immediate attention, including:\n- Suspected fraud or unauthorized account access\n- Security breaches or identity theft concerns\n- Large unexplained account balance changes\n- Inquiries about deceased individuals' accounts\n- Multiple failed authentic

In [22]:
improved_result

{'root_cause': 'The root cause analysis identified several issues: 1) Security issues were often misclassified as CARD_DISPUTE instead of ESCALATION. 2) General financial inquiries were incorrectly categorized into specific categories instead of IN_SCOPE. 3) There was no guidance for handling inquiries with multiple issues. 4) The template failed to recognize urgency or severity in certain contexts. These issues stemmed from ambiguous category definitions, a missing priority framework, incomplete classification criteria, and gaps in classification logic.',
 'improved_template': 'You are a Financial Services Assistant. Classify each customer inquiry into one of these categories:\n\nESCALATION - For urgent issues requiring immediate attention, including:\n- Suspected fraud or unauthorized account access\n- Security breaches or identity theft concerns\n- Large unexplained account balance changes\n- Inquiries about deceased individuals\' accounts\n- Multiple failed authentication attempts\

In [23]:
improved_template = improved_result['improved_template']
print(improved_template)

You are a Financial Services Assistant. Classify each customer inquiry into one of these categories:

ESCALATION - For urgent issues requiring immediate attention, including:
- Suspected fraud or unauthorized account access
- Security breaches or identity theft concerns
- Large unexplained account balance changes
- Inquiries about deceased individuals' accounts
- Multiple failed authentication attempts
- Urgent issues affecting immediate financial access

CARD_DISPUTE - For transaction-specific issues, including:
- Unauthorized or unrecognized charges
- Duplicate transactions
- Incorrect transaction amounts
- Merchandise or service disputes
- Missing refunds

TRANSACTION_STATUS - For questions about pending or completed financial movements:
- Payment processing timeframes
- Wire or transfer confirmations
- Deposit availability
- Check clearing status

AUTHENTICATION_SETUP - For establishing or modifying security credentials:
- Setting up biometric authentication
- Configuring multi-fac

### Test with the improved prompt template

In [24]:
# Create output filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = os.path.join(results_dir, f"test_results_improved_{timestamp}.json")

updated_test_cases = copy.deepcopy(test_cases)
updated_test_cases["prompt_template"] = improved_template
# Initialize counters and results structures
improved_result = execute_test_cases(updated_test_cases, target_model_id, output_file)

Processing Test Cases:   4%|▍         | 1/25 [00:01<00:30,  1.29s/it, Success=1/25]

{
  "prediction": "PIN_RESET",
  "explanation": "The primary request is to change the PIN for the card, which falls under PIN_RESET. The secondary request to update the mobile number is CONTACT_INFO_UPDATE, but since PIN_RESET is the main and urgent request, it takes priority."
}



Processing Test Cases:   8%|▊         | 2/25 [00:02<00:26,  1.17s/it, Success=2/25]

{
  "prediction": "ESCALATION",
  "explanation": "The inquiry involves multiple failed authentication attempts and a suspicious text message claiming to be from the bank, which raises concerns about potential fraud or unauthorized account access. These are urgent security issues that require immediate attention."
}



Processing Test Cases:  12%|█▏        | 3/25 [00:03<00:23,  1.07s/it, Success=3/25]

...

{
  "prediction": "IN_SCOPE",
  "explanation": "The inquiry is a standard financial service request for documentation to verify the source of funds for a down payment. It does not involve security concerns, transaction disputes, or authentication issues, but rather falls under general financial documentation requests."
}



Processing Test Cases:  96%|█████████▌| 24/25 [00:25<00:01,  1.00s/it, Success=24/25]

{
  "prediction": "ESCALATION",
  "explanation": "The sudden and significant drop in the credit score without any apparent reason could indicate a potential security issue, such as unauthorized account activity or identity theft. This requires immediate investigation to ensure the customer's financial security."
}



Processing Test Cases: 100%|██████████| 25/25 [00:26<00:00,  1.05s/it, Success=25/25]

{
  "prediction": "OUT_OF_SCOPE",
  "explanation": "The inquiry appears to be nonsensical and does not relate to any financial service or standard inquiry category. It seems to be a string of random characters and words without any coherent meaning or request."
}

Results saved to results/test_results_improved_20250312_093912.json


In [25]:
# Process each test case within the suite
task_success = 0
for index, test_case in enumerate(improved_result['test_cases']):
    # Evaluate with compare_result    
    if improved_result['test_cases'][index]['task_succeed'] == True:
        task_success +=1
        
improved_result['stats']['task_succeed'] = task_success
print("Successful: ", task_success, "of total number of", len(improved_result['test_cases']))

Successful:  18 of total number of 25
